# Scrapping Portal Inmobiliario

In [2]:
#importar librerías a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import sleep
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

import warnings
warnings.filterwarnings('ignore')

In [22]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.62 Safari/537.36")
#opts.add_argument("--headless")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
)

driver.get('https://www.portalinmobiliario.com/venta/departamento/proyectos')

#espera de carga de página por xx seg
wait = WebDriverWait(driver, 60)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
sleep(random.uniform(10.0, 14.0))

#Cantidad de páginas por búsqueda
#pagination = driver.find_element(By.XPATH, '//li[@class="andes-pagination__page-count"]').text
#pagination = int(pagination.split()[1])

#matriz para guardar df de iteraciones
records = []

#scrapping
print("Iniciando extracción de datos...")

for page in range(1, 2):

    #busca la moneda en la cual está publicado el anuncio
    monedas = driver.find_elements(By.XPATH, '//span[@class="andes-money-amount__currency-symbol"]')
    monedas = [ moneda.text for moneda in monedas ]
    monedas = monedas[5:]

    #busca el precio de cada publicación
    precios = driver.find_elements(By.XPATH,'//span[@class="andes-money-amount__fraction"]')
    precios = [precio.text for precio in precios]
    precios = precios[5:]

    #Extrae m2 del depto
    m2s = driver.find_elements(By.XPATH, '//ul[@class="ui-search-card-attributes ui-search-item__group__element ui-search-item__attributes-grid"]//li[3]')
    m2s = [m2.text for m2 in m2s]
    m2s = m2s[5:]

    #extrae nro de piezas
    dormitorios = driver.find_elements(By.XPATH, '//ul[@class="ui-search-card-attributes ui-search-item__group__element ui-search-item__attributes-grid"]//li[1]')
    dormitorios = [dormitorio.text for dormitorio in dormitorios]
    dormitorios = dormitorios[5:]

    #extrae nr de baños
    baños = driver.find_elements(By.XPATH, '//ul[@class="ui-search-card-attributes ui-search-item__group__element ui-search-item__attributes-grid"]//li[2]')
    baños = [baño.text for baño in baños]
    baños = baños[5:]

    #extrae direccion
    direcciones = driver.find_elements(By.XPATH, '//p[@class="ui-search-item__group__element ui-search-item__location-label"]')
    direcciones = [direccion.text for direccion in direcciones]
    direcciones = direcciones[5:]

    #extrae links de proyectos
    links = driver.find_elements(By.XPATH, '//div[@class="andes-carousel-snapped__slide andes-carousel-snapped__slide--active"]//a')
    links = [link.get_attribute('href') for link in links]

    #creación de dataframe y agregado al records
    datos = {
        "moneda" : monedas,
        "precio" : precios,
        "tamaño" : m2s,
        "dormitorios" : dormitorios,
        "baños" : baños,
        "direccion" : direcciones,
        "link" : links
    }
    df = pd.DataFrame(datos)

    records.append(df)


    #botón página siguiente
    next_page_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@title="Siguiente"]'))
    )

    next_page_button.click()
    
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    sleep(random.uniform(10.0, 14.0))

driver.quit()


df = pd.concat(records)

df.to_csv("proyectos.csv")

Iniciando extracción de datos...


ValueError: All arrays must be of the same length

In [23]:
dormitorios

['1 a 3 dormitorios',
 '1 a 3 dormitorios',
 '2 a 3 dormitorios',
 '1 a 3 dormitorios',
 '1 a 2 dormitorios',
 '1 a 3 dormitorios',
 '1 a 3 dormitorios',
 '1 a 2 dormitorios',
 '1 a 2 dormitorios',
 '1 a 2 dormitorios',
 '2 a 3 dormitorios',
 '1 a 2 dormitorios',
 '1 a 2 dormitorios',
 '2 dormitorios',
 '1 a 2 dormitorios',
 '1 a 2 dormitorios',
 '1 a 2 dormitorios',
 '1 a 3 dormitorios',
 '2 a 3 dormitorios',
 '1 a 4 dormitorios',
 '1 a 3 dormitorios',
 '1 a 4 dormitorios',
 '4 dormitorios',
 '1 a 3 dormitorios',
 '1 a 3 dormitorios',
 '4 a 5 dormitorios',
 '1 a 3 dormitorios',
 '1 a 3 dormitorios',
 '1 a 3 dormitorios',
 '1 a 3 dormitorios',
 '1 a 3 dormitorios',
 '3 a 4 dormitorios',
 '2 a 3 dormitorios',
 '1 a 3 dormitorios',
 '1 a 2 dormitorios',
 '2 a 4 dormitorios',
 '1 a 3 dormitorios',
 '1 a 2 dormitorios',
 '2 a 3 dormitorios',
 '1 a 3 dormitorios',
 '2 a 4 dormitorios',
 '2 a 3 dormitorios',
 '2 a 3 dormitorios']

In [19]:
datos


{'moneda': ['UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF',
  'UF'],
 'precio': ['2.550',
  '2.590',
  '2.965',
  '2.621',
  '1.809',
  '5.568',
  '4.430',
  '9.390',
  '2.290',
  '4.183',
  '2.290',
  '3.680',
  '3.390',
  '1.569',
  '2.400',
  '5.056',
  '2.605',
  '3.308',
  '7.000',
  '1.884',
  '3.520',
  '4.199',
  '6.988',
  '11.544',
  '1.890',
  '6.522',
  '8.049',
  '13.679',
  '2.490',
  '6.040',
  '14.027',
  '3.350',
  '2.890',
  '2.978',
  '2.976',
  '13.508',
  '12.090',
  '9.867',
  '6.750',
  '4.004',
  '8.800',
  '2.355',
  '2.682',
  '4.150',
  '6.450',
  '12.221',
  '9.791',
  '7.065'],
 'tamaño': [],
 'dormitorios': ['35 a 68 m² útiles',
  '1 a 